In [238]:
### Calculate (future) ACE from forecasts of a single intensity model using cyclone a-tracks
### Uses linear interpolation to get values of 6h intervals

# download latest a-tracks manually for basin of interest (no automation yet)
# and change names of atrack files below

# note: make sure to gunzip them
# https://ftp.nhc.noaa.gov/atcf/aid_public/?C=M;O=D
    
atrack_files = [
    'aal132023.dat',
    'aal142023.dat',
    'aal972023.dat'
]

##### CONFIG BASIN AND INTENSITY MODEL
match_basin = 'AL'
# consensus model
match_model = 'IVCN'


In [239]:
ab_column_names = [
    'BASIN',
    'CY',
    'YYYYMMDDHH',
    'TECHNUM/MIN',
    'TECH',
    'TAU',
    'LatN/S',
    'LonE/W',
    'VMAX',
    'MSLP',
    'TY',
    'RAD',
    'WINDCODE',
    'RAD1',
    'RAD2',
    'RAD3',
    'RAD4',
    'POUTER',
    'ROUTER',
    'RMW',
    'GUSTS',
    'EYE',
    'SUBREGION',
    'MAXSEAS',
    'INITIALS',
    'DIR',
    'SPEED',
    'STORMNAME',
    'DEPTH',
    'SEAS',
    'SEASCODE',
    'SEAS1',
    'SEAS2',
    'SEAS3',
    'SEAS4',
    'USERDEFINED1',
    'userdata1',
    'USERDEFINED2',
    'userdata2',
    'USERDEFINED3',
    'userdata3',
    'USERDEFINED4',
    'userdata4',
    'USERDEFINED5',
    'userdata5'
]

In [240]:
import pandas as pd
from datetime import datetime

track_line_lists = []

# read the tracks
for atrack_file in atrack_files:
    with open(atrack_file, 'r') as file:
        lines = file.readlines()
    for line in lines:
        line_list = line.split(',')
        line_list_trim = [x.strip() for x in line_list]
        line_list_trim_rempty = line_list_trim
        
        # trim empty columns on right
        # a tracks have an extra comma before EOL for some reason (should have max 45 columns)
        line_list_trim_rempty.reverse()
        line_list_trim_empty = []
        for [i, v] in enumerate(line_list_trim_rempty):
            if v:
                line_list_trim_empty = line_list_trim_rempty[i:]
                break
        line_list_trim_empty.reverse()
        # convert init time strings to timestamp format
        line_list_trim_empty[2] = datetime.strptime(line_list_trim_empty[2], '%Y%m%d%H')
            
        track_line_lists.append(line_list_trim_empty)

In [241]:
# convert the line lists to a dataframe
df = pd.DataFrame(track_line_lists, columns=ab_column_names)

In [242]:
# get the last matching forecasts
df_match = df.loc[(df['BASIN'] == match_basin) & (df['TECH'] == match_model)]

newest_matching_cyclone_forecast_dfs = []
for cyclone_num in df_match['CY'].unique():
    df_cyclone = df_match.loc[(df['CY'] == cyclone_num)]
    newest_init_time = df_cyclone['YYYYMMDDHH'].max()
    newest_forecast = df_cyclone.loc[(df['YYYYMMDDHH'] == newest_init_time)]
    # make sure these columns are numeric
    # convert to numeric
    col_numeric_names = ["CY", "TAU", "VMAX", "MSLP"]
    for col_numeric_name in col_numeric_names:
        newest_forecast = newest_forecast.astype({col_numeric_name:'int'})
    # append to list of cyclone forecast dfs
    newest_matching_cyclone_forecast_dfs.append(newest_forecast)

In [243]:
import datetime as datetime
# interpolate to provide 06 taus
interp_intensity_cyclone_dfs = []

for cyclone_df in newest_matching_cyclone_forecast_dfs:
    forecast_taus = cyclone_df['TAU'].unique()
    min_tau = forecast_taus.min()
    max_tau = forecast_taus.max()
    taus_6h = list(range(min_tau, max_tau + 6, 6))
    taus_missing = []
    for tau in taus_6h:
        if tau not in forecast_taus:
            taus_missing.append(tau)
    for missing_tau in taus_missing:
        # get prev tau
        prev_tau = missing_tau
        while prev_tau >= min_tau:
            prev_tau = prev_tau - 1
            if prev_tau in forecast_taus:
                break
        next_tau = missing_tau
        while next_tau <= max_tau:
            next_tau = next_tau + 1
            if next_tau in forecast_taus:
                break
        prev_intensity = cyclone_df.loc[(cyclone_df['TAU'] == prev_tau)]['VMAX'].values[0]
        next_intensity = cyclone_df.loc[(cyclone_df['TAU'] == next_tau)]['VMAX'].values[0]
        interpolated_intensity = int(round((prev_intensity + next_intensity) / 2.0))
        cyclone_df = cyclone_df.reset_index(drop=True)
        prev_idx = cyclone_df.loc[(cyclone_df['TAU'] == prev_tau)].index.values[0]
        # copy row
        interp_row = cyclone_df.iloc[prev_idx].copy()
        # only manipulate the TAU and INTENSITY so we don't have to copy the rest of the values
        # the long/lat and rest of data have not been interpolated....
        interp_row['TAU'] = missing_tau
        interp_row['VMAX'] = interpolated_intensity
        # add row
        #cyclone_df = cyclone_df.append(interp_row, ignore_index=True)
        cyclone_df = pd.concat([cyclone_df, pd.DataFrame([interp_row])], ignore_index=True)
    cyclone_df = cyclone_df.reset_index(drop=True)
    cyclone_df.sort_values(by=['TAU'], inplace=True)
    cyclone_df = cyclone_df.reset_index(drop=True)
    interp_intensity_cyclone_dfs.append(cyclone_df)
    # cyclone number
    cyclone_num = cyclone_df['CY'].iloc[-1]
    basin = cyclone_df.iloc[-1]['BASIN']
    init_time_str = cyclone_df.iloc[0]['YYYYMMDDHH']
    valid_time_str = init_time_str + datetime.timedelta(hours=int(max_tau))
    # print the forecast times
    print(f"{basin}{cyclone_num}: 0-{max_tau}H : {init_time_str} to {valid_time_str}")

AL13: 0-132H : 2023-09-14 00:00:00 to 2023-09-19 12:00:00
AL14: 0-168H : 2023-09-14 00:00:00 to 2023-09-21 00:00:00
AL97: 0-132H : 2023-09-14 00:00:00 to 2023-09-19 12:00:00


In [244]:
# exclude storms less than 35 kt
# exclude forecasts at 00h (since this may already be included in real time ACE)
ACE_min_kt = 35
storm_intensities_above_min = {}
for cyclone_df in interp_intensity_cyclone_dfs:
    cyclone_num = cyclone_df['CY'].iloc[-1]
    basin = cyclone_df.iloc[-1]['BASIN']
    storm_name_str = f'{basin}{cyclone_num}'
    min_kt_rows = cyclone_df.loc[(cyclone_df['VMAX'] >= ACE_min_kt) & (cyclone_df['TAU'] > 0)]['VMAX']
    storm_intensities_above_min[storm_name_str] = list(min_kt_rows.values)

In [245]:
import numpy as np
forecasts_ace = {}
for [storm_name, vmax] in storm_intensities_above_min.items():
    forecast_storm_ace = (pow(10, -4) * np.sum(np.power(vmax, 2)))
    forecasts_ace[storm_name] = round(forecast_storm_ace,2)

In [246]:
forecast_total_ace = 0
print(f"Future contributions of cyclone ACE values in {match_basin} basin (including INVESTs)\n",
      f"   from latest forecast using {match_model} (excluding 00h) by interpolating intensities")
for [storm_name, ace] in forecasts_ace.items():
    print(f"  ACE(future) for {storm_name} = {ace}")
    forecast_total_ace += ace
print(f"  Total ACE(future): {forecast_total_ace:4.2f}")

Future contributions of cyclone ACE values in AL basin (including INVESTs)
    from latest forecast using IVCN (excluding 00h) by interpolating intensities
  ACE(future) for AL13 = 7.3
  ACE(future) for AL14 = 7.12
  ACE(future) for AL97 = 11.32
  Total ACE(future): 25.74


In [247]:
print(storm_intensities_above_min)

{'AL13': [87, 84, 82, 80, 78, 75, 74, 74, 68, 62, 54, 47, 41, 35, 35, 37, 40, 43], 'AL14': [72, 69, 68, 66, 64, 61, 60, 58, 56, 53, 50, 48, 47, 46, 48, 50, 52, 55, 53, 51, 50, 48, 41], 'AL97': [37, 39, 41, 45, 49, 55, 61, 66, 72, 79, 86, 95, 104, 109, 114, 119, 124]}


In [248]:
print(interp_intensity_cyclone_dfs)

[   BASIN  CY YYYYMMDDHH TECHNUM/MIN  TECH  TAU LatN/S LonE/W  VMAX  MSLP  ...  \
0     AL  13 2023-09-14          03  IVCN    0     0N     0W    90     0  ...   
1     AL  13 2023-09-14          03  IVCN    6     0N     0W    87     0  ...   
2     AL  13 2023-09-14          03  IVCN   12     0N     0W    84     0  ...   
3     AL  13 2023-09-14          03  IVCN   18     0N     0W    82     0  ...   
4     AL  13 2023-09-14          03  IVCN   24     0N     0W    80     0  ...   
5     AL  13 2023-09-14          03  IVCN   30     0N     0W    78     0  ...   
6     AL  13 2023-09-14          03  IVCN   36     0N     0W    75     0  ...   
7     AL  13 2023-09-14          03  IVCN   42     0N     0W    74     0  ...   
8     AL  13 2023-09-14          03  IVCN   48     0N     0W    74     0  ...   
9     AL  13 2023-09-14          03  IVCN   54     0N     0W    68     0  ...   
10    AL  13 2023-09-14          03  IVCN   60     0N     0W    62     0  ...   
11    AL  13 2023-09-14    